# Exercise 4. Backtracking algorithms
#### Algorithms ans Data Structures

by Jędrzej Kopiszka, 145304
10.06.2020

In [123]:
from timeit import default_timer as timer #time measuring
import concurrent.futures as futures #setting maximum execution time
import pandas as pd # DataFrame for output representation
import numpy as np 
import random # randomized element search
import seaborn as sns # plotting
import matplotlib.pyplot as plt #plotting

## 1. Graph representation, connectivity check
We can store graphs in different representations. For this exercise I use two methods: edge list in format [[a,b],[c,d]] and neighborhood matrix

#### 1.1 Algorithms for different representation

In [124]:
#list of incidents
def list_of_incidents(directed, size, tab):
    final_tab=[[] for i in range(size)]
    if directed==True:
        for vertex in tab:
            final_tab[vertex[0]].append(vertex[1])
          
    else:
        for vertex in tab:
            final_tab[vertex[0]].append(vertex[1])
            final_tab[vertex[1]].append(vertex[0])
            
    return final_tab

In [125]:
#edge list
def edge_list(directed, size, tab):
    final_tab=[]
    if directed==True:
        final_tab=tab
    else:
        for vertex in tab:
            final_tab.append(vertex)
            final_tab.append([vertex[1], vertex[0]])
    return final_tab

In [126]:
#Neighborhood matrix (vertex matrix, adjacency matrix)
def neighborhood_matrix(directed, size, tab):
    final_tab = [[0 for j in range(size)] for i in range(size)]
    if directed==False:
        for vertex in tab:
            final_tab[vertex[0]][vertex[1]]=1
            final_tab[vertex[1]][vertex[0]]=1
    else:
        for vertex in tab:
            final_tab[vertex[0]][vertex[1]]=1   
    return final_tab

In [127]:
#Incident matrix
def incident_matrix(directed, size, tab):
    final_tab=[[0 for i in range(len(tab))] for i in range(size)]
    if directed==False:
        edge_num=0
        for vertex in tab:
            final_tab[vertex[0]][edge_num]=1
            final_tab[vertex[1]][edge_num]=1
            edge_num+=1
    else:
        edge_num=0
        for vertex in tab:
            final_tab[vertex[0]][edge_num]=-1
            final_tab[vertex[1]][edge_num]=1
            edge_num+=1
    return final_tab

#### 1.2 Neighborhood matrix to edge list converter

In [128]:
def neighborhood_to_list_of_edges(neighborhood):
    edge=[]
    for i in range(len(neighborhood)):
        for j in range(len(neighborhood)):
            if neighborhood[i][j]==1:
                edge.append([i,j])
    return edge

#### 1.3 Connectivity check - if graph is connected

In [129]:
#checking connectivity using list of edges representation
def check_connectivity(edges,n):
    visited = [0 for i in range(n)]
    s=[]
    vc=0
    s.insert(0,0)
    visited[0]=1
    while(len(s)!=0):
        v=s[0]
        s.pop(0)
        vc+=1
        for edge in edges:
            if edge[0]==v:
                if visited[edge[1]]==1:
                    continue
                else:
                    visited[edge[1]]=1
                    s.insert(0,edge[1])
            elif edge[1]==v:
                if visited[edge[0]]==1:
                    continue
                else:
                    visited[edge[0]]=1
                    s.insert(0,edge[0])
    if vc==n:
        return True
    else:
        return False

## 2. Eulerian circuit

#### 2.1 Generate connected graphs with eulerian cycle for different saturations

In [130]:
def generator_eulerian(n,saturation):
    verticies = list(np.arange(n))
    start = random.choice(verticies)
    verticies.remove(start)
    path=[start]
    for i in range(n-1):
        choice = random.choice(verticies)
        verticies.remove(choice)
        path.append(choice)
    path.append(start)
    edges=[]
    for i in range(len(path)-1):
        edges.append([path[i], path[i+1]])
    neighborhood = neighborhood_matrix(False, n, edges) 
    l=n
    stop=False
    for i in range(n):
        for j in range(i+1,n):
            for z in range(j+1, n):
                if neighborhood[i][j]!=1 and neighborhood[j][z]!=1 and neighborhood[z][i]!=1:
                    neighborhood[i][j]=1
                    neighborhood[j][i]=1
                    neighborhood[j][z]=1
                    neighborhood[z][j]=1
                    neighborhood[z][i]=1
                    neighborhood[i][z]=1
                    l+=3
                if  l>=(saturation*n*(n-1)*0.5):
                    stop=True
                    break
            if stop==True:
                break
        if stop==True:
            break         
    return neighborhood

#### 2.2 Find eulerian path - Fleury’s Algorithm

In [131]:
#find eulerian circuit
def find_start_vert(tmpGraph, n):
    for i in range(n):
        deg=0
        for j in range(n):
            if tmpGraph[i][j]==1:
                deg+=1
        if deg%2!=0:
            return i
    return 0

def dfs(prev,start,visited, tmpGraph, n):
    count=1
    visited[start]=True
    for u in range(n):
        if prev!=u:
            if visited[u]!=1:
                if tmpGraph[start][u]==1:
                    count+=dfs(start, u, visited, tmpGraph,n)
    return count

def isBridge(u,v,tmpGraph,n):
    deg=0
    for i in range(n):
        if tmpGraph[v][i]==1:
            deg+=1
    if deg>1:
        return False
    return True

def edgeCount(tmpGraph, n):
    count=0
    for i in range(n):
        for j in range(n):
            if tmpGraph[i][j]==1:
                count+=1
    return count

def fleury_algorithm(start,tmpGraph, n):
    edge=edgeCount(tmpGraph,n)
    v_count = n
    for v in range(n):
        if tmpGraph[start][v]==1:
            visited = [0 for i in range(n)] 
            if isBridge(start,v, tmpGraph, n)==1:
                v_count-=1
            cnt = dfs(start, v, visited, tmpGraph, n)
            if abs(v_count-cnt)<=2:
                return True
                if isBridge(v,start, tmpGraph, n)==1:
                    v_count-=1
                tmpGraph[start][v]=0
                tmpGraph[v][start]=0
                edge-=1
                fleury_algorithm(v, tmpGraph, n)

def find_eulerian_circuit(graph,n):
    tmpGraph = [[graph[i][j] for j in range(n)] for i in range(n)]
    fleury_algorithm(find_start_vert(tmpGraph, n), tmpGraph, n)

#### 2.3 Generate data for time-measuring

In [132]:
def genertae_data_for_eulerian():
    list_of_saturation=[0.2, 0.3, 0.4, 0.6, 0.8, 0.95]
    dict_of_eulerian_graphs = {}
    for n in list(np.arange(50,401, 50)):
        dict_of_eulerian_graphs[n]=[]
        print("generating for n= ",n)
        for i in range(len(list_of_saturation)): #generate saturation
            dict_of_eulerian_graphs[n].append([])
            print("10 graphs with saturation:", end=' ')
            how_many=0
            while(how_many<10):
                graph = generator_eulerian(n,list_of_saturation[i]) #generate random graph, check connectivity
                if check_connectivity(neighborhood_to_list_of_edges(graph), n)==1:
                    dict_of_eulerian_graphs[n][i].append(graph) # add generated graph to dictionary
                    how_many+=1
            print(list_of_saturation[i], end=', ')
        print('\n')
    return dict_of_eulerian_graphs

In [133]:
dict_of_eulerian = genertae_data_for_eulerian()

generating for n=  50
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs with saturation: 0.6, 10 graphs with saturation: 0.8, 10 graphs with saturation: 0.95, 

generating for n=  100
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs with saturation: 0.6, 10 graphs with saturation: 0.8, 10 graphs with saturation: 0.95, 

generating for n=  150
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs with saturation: 0.6, 10 graphs with saturation: 0.8, 10 graphs with saturation: 0.95, 

generating for n=  200
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs with saturation: 0.6, 10 graphs with saturation: 0.8, 10 graphs with saturation: 0.95, 

generating for n=  250
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs 

#### 2.4 Measure the time of finding eulerian cycle in connected graph for different sizes and saturations

In [134]:
times_eulerian={}
for key in dict_of_eulerian.keys():
    times_eulerian[key]=[]
    print("\n Done for key", key, "saturation: ", end='')
    itera=0
    for saturation in dict_of_eulerian[key]:
        mean_time=0
        itera+=1
        for graph in saturation:
            def test():
                    start=timer()
                    a = find_eulerian_circuit(graph,key)
                    end = timer()
                    return end-start
            with futures.ThreadPoolExecutor(max_workers=1) as executor:
                future = executor.submit(test)
                try:
                    resp = future.result(60*5)
                except futures.TimeoutError:
                    mean_time+=60*5
                else:
                    mean_time+=resp
                executor._threads.clear()
                futures.thread._threads_queues.clear()
        mean_time/=10
        times_eulerian[key].append(mean_time)
        print(itera, end=", ")


 Done for key 50 saturation: 1, 2, 3, 4, 5, 6, 
 Done for key 100 saturation: 1, 2, 3, 4, 5, 6, 
 Done for key 150 saturation: 1, 2, 3, 4, 5, 6, 
 Done for key 200 saturation: 1, 2, 3, 4, 5, 6, 
 Done for key 250 saturation: 1, 2, 3, 4, 5, 6, 
 Done for key 300 saturation: 1, 2, 3, 4, 5, 6, 
 Done for key 350 saturation: 1, 2, 3, 4, 5, 6, 
 Done for key 400 saturation: 1, 2, 3, 4, 5, 6, 

#### 2.5 Execution times - dataframe and plot

In [135]:
df_eulerian = pd.DataFrame.from_dict(orient='index',data=times_eulerian, columns=[0.2,0.3,0.4,0.6,0.8,0.95])
df_eulerian

,0.2,0.3,0.4,0.6,0.8,0.95
50,0.000938,0.000930,0.000886,0.000826,0.000751,0.000689
100,0.002243,0.002412,0.002342,0.002784,0.003141,0.003020
150,0.004818,0.004972,0.005808,0.005374,0.005592,0.005697
200,0.009755,0.009067,0.008906,0.010018,0.009844,0.011040
250,0.013259,0.013689,0.013939,0.016076,0.014687,0.016059
300,0.019492,0.020101,0.022261,0.021398,0.022034,0.023463
350,0.028138,0.026946,0.027954,0.028303,0.031286,0.031693
400,0.034799,0.035620,0.037723,0.040295,0.039963,0.040255


#### 2.6 Conclusions on graph representation

In the analysis, I use two graph representation methods: edge list and neighborhood matrix. 
Both methods are easily understandable and the most popular- unqualified readers should have no problem with understanding the representation or finding information about them - that is an advantage over the "incident matrix" and "list of incidents" whose representation may be confusing.

When it comes to performance - the biggest difference is in accessing or iterating through the representation when checking if a specific pair of vertices is already an edge. Edge list is accessed linearly meaning each time we need to go through the whole list to find if the list contains that pair. In neighborhood matrix, we can access specific pair by [i][j]==1 or [i][j]==0. That is much faster, especially works much better in generating random graphs which is done partially by looking for a pair that does not have an edge.

## 3. Hamiltonian cycle

#### 3.1  Random, connected graph with varius saturation - generator function

In [136]:
#generate connected for checking if is hamiltonian
def generate_connected(n,saturation):
    verticies = list(np.arange(n))
    start = random.choice(verticies)
    verticies.remove(start)
    path=[start]
    for i in range(n-1):
        choice = random.choice(verticies)
        verticies.remove(choice)
        path.append(choice)
    path.append(start)
    edges=[]
    for i in range(len(path)-1):
        edges.append([path[i], path[i+1]]) 
    every_permutation=[]
    for i in range(n):
        for j in range(i, n):
            if i!=j:
                every_permutation.append([i,j])
    while(len(edges)<((saturation*n*(n-1))/2)):
        a=random.choice(every_permutation)
        if a not in edges and [a[1],a[0]] not in edges:
            edges.append(a)
            every_permutation.remove(a)
            
    return edges

#### 3.2 Finding Hamiltonian cycle - backtracking algorithm

In [138]:
#checks if contains hamiltonian
def isValid(v,k, graph, path):
    if graph[path[k-1]][v]==0:
        return False
    for i in range(k):
        if path[i]==v:
            return False
    return True

def cycle_found(k, graph,n, path):
    if k==n:
        if graph[path[k-1]][path[0]]==1:
            return True
        else:
            return False
    for v in range(n):
        if isValid(v,k, graph,path)==True:
            path[k] = v
            if cycle_found(k+1, graph,n, path)==True:
                return True
            path[k]=-1
    return False


def hamilton_cycle(graph,n):
    path=[0 for i in range(n)]
    for i in range(n):
        path[i]=-1
    path[0]=0
    if cycle_found(1, graph,n, path)==False:
        return "yes"
    else:
        return "no"
    

#### 3.3 Generating random, connected graphs with different saturation and size, 10 graphs each combination

In [142]:
def genertae_data_for_hamilton():
    list_of_saturation=[0.2, 0.3, 0.4, 0.6, 0.8, 0.95]
    dict_of_connected_graphs = {}
    for n in list(np.arange(50,311, 20)):
        dict_of_connected_graphs[n]=[]
        print("generating for n= ",n)
        for i in range(len(list_of_saturation)): #generate saturation
            dict_of_connected_graphs[n].append([])
            print("10 graphs with saturation:", end=' ')
            how_many=0
            while(how_many<10):
                graph = generate_connected(n,list_of_saturation[i]) #generate random graph
                if check_connectivity(graph, n)==True: #check connectivity of graph
                    graph = neighborhood_matrix(False, n, graph) #change representation to neighborhood matrix
                    dict_of_connected_graphs[n][i].append(graph) # add generated graph to dictionary
                    how_many+=1
            print(list_of_saturation[i], end=', ')
        print('\n')
    return dict_of_connected_graphs

In [139]:
dict_of_graphs = genertae_data_for_hamilton() #generate data using the function above

generating for n=  50
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs with saturation: 0.6, 10 graphs with saturation: 0.8, 10 graphs with saturation: 0.95, 

generating for n=  70
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs with saturation: 0.6, 10 graphs with saturation: 0.8, 10 graphs with saturation: 0.95, 

generating for n=  90
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs with saturation: 0.6, 10 graphs with saturation: 0.8, 10 graphs with saturation: 0.95, 

generating for n=  110
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs with saturation: 0.6, 10 graphs with saturation: 0.8, 10 graphs with saturation: 0.95, 

generating for n=  130
10 graphs with saturation: 0.2, 10 graphs with saturation: 0.3, 10 graphs with saturation: 0.4, 10 graphs wi

#### 3.4 Time-measuring for finding Hamiltonian cycle 

In [140]:
times_hamiltonian={}
for key in dict_of_graphs.keys():
    times_hamiltonian[key]=[]
    print("Done for key", key, "saturation: ", end='')
    itera=0
    for saturation in dict_of_graphs[key]:
        mean_time=0
        itera+=1
        for graph in saturation:
            def test():
                    start=timer()
                    a = hamilton_cycle(graph,key)
                    end = timer()
                    return end-start
            with futures.ThreadPoolExecutor(max_workers=1) as executor:
                future = executor.submit(test)
                try:
                    resp = future.result(60*2)
                except futures.TimeoutError:
                    mean_time+=60*2
                else:
                    mean_time+=resp
                executor._threads.clear()
                futures.thread._threads_queues.clear()
        mean_time/=10
        times_hamiltonian[key].append(mean_time)
        print(itera, end=", ")
    print('\n')

Done for key 50 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 70 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 90 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 110 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 130 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 150 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 170 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 190 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 210 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 230 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 250 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 270 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 290 saturation: 1, 2, 3, 4, 5, 6, 

Done for key 310 saturation: 1, 2, 3, 4, 5, 6, 



#### 3.5 Execution time - dataframe and plot

In [141]:
df_hamiltonian = pd.DataFrame.from_dict(orient='index',data=times_hamiltonian, columns=[0.2,0.3,0.4,0.6,0.8,0.95])
df_hamiltonian

,0.2,0.3,0.4,0.6,0.8,0.95
50,20.179704,7.978367,5.167325,0.007590,0.001163,0.001294
70,25.366296,12.142463,5.586654,0.090305,0.003315,0.002772
90,21.529735,27.001540,5.312274,0.257054,0.014308,0.109961
110,30.000000,22.761337,5.804537,0.344063,0.032080,0.397452
130,26.033934,16.779127,10.088534,0.374733,0.057354,0.349705
150,27.001313,24.957734,10.123400,5.174613,1.045384,1.402749
170,30.000000,22.280176,18.755221,2.036097,2.172929,1.974654
190,30.000000,23.805221,20.052528,2.519742,2.379237,3.379856
210,27.463535,26.245347,19.850652,6.972508,4.910597,5.849585
230,30.000000,25.973047,22.167896,9.801155,7.742113,9.050664


## 4. Conclusion regarding complexity and algorithms

Finding Euler cycle: For finding Euler circuit I used Fleury's Algorithm. The idea behind the algorithm is straightforward: start in selected vertex, next repeat: add "u" to the solution, select [u,v] and remove it (do not remove bridge if possible). I have chosen this algorithm mainly because of easy-to-understand steps, simple implementation. The downside of this algorithm is it's high running time - O(|E|^2) (we can improve it by using advanced Thorup algorithm - then O(|E|(log|E|^3) ). Another idea is to use Hierholzer's algorithm - O(|E|)

Finding Hamilton cycle: I used a backtracking approach. It's idea is to start from any vertex, we make selected vertex a root of the constructed tree. Then we select the next vertex. When selected vertex makes a cycle with any vertex other than root we reach the maximum step. We must backtrack one step, and start the search once again by selecting another vertex and backtrack and remove the successful path. Such approach can give us polynomial time complexity for the NP-hard problem.